In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq playwright==1.46.0 --progress-bar off
!pip install -qqq html2text==2024.2.26 --progress-bar off
!playwright install chromium

  Preparing metadata (setup.py) ... done
162.8 MiB [] 0% 10.8s162.8 MiB [] 0% 16.1s162.8 MiB [] 0% 16.4s162.8 MiB [] 0% 10.6s162.8 MiB [] 0% 8.4s162.8 MiB [] 1% 7.7s162.8 MiB [] 1% 6.9s162.8 MiB [] 1% 6.1s162.8 MiB [] 2% 5.9s162.8 MiB [] 2% 5.8s162.8 MiB [] 3% 5.6s162.8 MiB [] 3% 5.0s162.8 MiB [] 4% 4.7s162.8 MiB [] 4% 4.8s162.8 MiB [] 5% 4.6s162.8 MiB [] 5% 4.3s162.8 MiB [] 6% 4.2s162.8 MiB [] 6% 4.0s162.8 MiB [] 6% 4.2s162.8 MiB [] 7% 4.0s162.8 MiB [] 8% 3.9s162.8 MiB [] 8% 3.8s162.8 MiB [] 9% 3.8s162.8 MiB [] 9% 3.7s162.8 MiB [] 10% 3.7s162.8 MiB [] 10% 3.6s162.8 MiB [] 11% 3.5s162.8 MiB [] 11% 3.6s162.8 MiB [] 12% 3.5s162.8 MiB [] 13% 3.5s162.8 MiB [] 13% 3.4s162.8 MiB [] 13% 3.5s162.8 MiB [] 14% 3.6s162.8 MiB [] 14% 3.7s162.8 MiB [] 15% 3.6s162.8 MiB [] 16% 3.5s162.8 MiB [] 16% 3.4s162.8 MiB [] 17% 3.4s162.8 MiB [] 18% 3.3s162.8 MiB [] 19% 3.1s162.8 MiB [] 20% 3.0s162.8 MiB [] 21% 2.9s162.8 MiB [] 22% 2.8s162.8 MiB [] 23% 2.6s162.8 MiB [] 24% 2.5s162.8 MiB [] 25% 2.5s162.8 MiB [] 

In [8]:
import requests
from bs4 import BeautifulSoup, NavigableString
import json
import re

def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip())

def scrape_events(base_url, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_items = soup.find_all('div', class_='eventitem')

    events = []

    for event in event_items:
        event_name = clean_text(event.find('h1').get_text(strip=True))
        event_date = clean_text(event.find('div', class_='eventdate').get_text(strip=True))

        category = clean_text(", ".join([term.get_text(strip=True) for term in event.find_all('div', class_='term')]))

        event_date_div = event.find('div', class_='eventdate')

        description = ''

        for sibling in event_date_div.next_siblings:
            if sibling.name == 'a' and 'button' in sibling.get('class', []):
                break
            elif isinstance(sibling, NavigableString):
                description += sibling.strip()
            elif sibling.name is not None:
                description += sibling.get_text(strip=True)

        description = clean_text(description)

        read_more_link_tag = event.find('a', class_='button', href=True)
        if read_more_link_tag:
            read_more_link = read_more_link_tag['href']
            full_read_more_url = read_more_link if read_more_link.startswith('http') else base_url + read_more_link

            more_details = scrape_more_details(full_read_more_url)
        else:
            more_details = {"venue": "", "more_description": ""}

        event_details = {
            "name": event_name,
            "date": event_date,
            "description": description,
            "category": category,
            "venue": more_details['venue'],
            "more_details": more_details['more_description']
        }
        events.append(event_details)

    return events

def scrape_more_details(more_url):
    response = requests.get(more_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    venue_section = soup.find('div', class_='eventlocation')
    venue = clean_text(venue_section.get_text(separator=' ', strip=True)) if venue_section else "N/A"

    description_section = soup.find('div', class_='copyContent expanded')
    if description_section:
        more_description = description_section.find_all('p')
        full_description = " ".join([clean_text(para.get_text(separator=' ', strip=True)) for para in more_description])
    else:
        full_description = ""

    return {"venue": venue, "more_description": full_description}

def convert_to_json(events):
    return json.dumps(events, indent=4)

base_url = 'https://downtownpittsburgh.com'

urls = [
    "https://downtownpittsburgh.com/events/?n=1&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=2&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=3&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=4&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=5&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=6&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=7&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=8&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=9&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=10&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=11&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=12&y=2024&cat=0",
    "https://downtownpittsburgh.com/events/?n=1&y=2025&cat=0",
    "https://downtownpittsburgh.com/events/?n=2&y=2025&cat=0"
]

all_events = []

for url in urls:
    events = scrape_events(base_url, url)
    all_events.extend(events)

event_json = convert_to_json(all_events)

print(event_json)

with open('events.json', 'w') as json_file:
    json_file.write(event_json)


[
    {
        "name": "Creative Conversations - My Fair Lady",
        "date": "Jan 31, 2024 | 6:30 pm - 7:30 pm",
        "description": "Get the inside scoop on the PNC Broadway in Pittsburgh performances with Creative Conversations presented by the Trust Arts Education Department.",
        "category": "Arts + Culture,, Workshops & Classes",
        "venue": "Trust Arts Education Center 807 Liberty Ave Pittsburgh, PA 15222",
        "more_details": ""
    },
    {
        "name": "Radiant Strings",
        "date": "Jan 31, 2024 | 6:00 pm - 8:00 pm",
        "description": "Radiant Strings is a Pittsburgh-based string ensemble comprised of classically trained musicians and is the ideal blend of classical refinement, modern edge and an infectious energy.",
        "category": "Free event,, Music",
        "venue": "Greer Cabaret Theater 655 Penn Ave Pittsburgh, PA 15222",
        "more_details": ""
    },
    {
        "name": "Best Restaurants",
        "date": "Jan 31, 2024 | 6:00

In [9]:
with open('events.json', 'r') as file:
    data = json.load(file)

number_of_entries = len(data)
print(f'The total number of entries is: {number_of_entries}')

The total number of entries is: 1883
